In [3]:
import asyncio
from typing import Dict, Any, List, TypedDict
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda, RunnableParallel
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.output_parsers import StrOutputParser
from langgraph.graph import StateGraph, END

# Assume these imports and functions are available
from your_modules import LLMGateway, run_vespa_search, get_vespa_payload, parse_json_safely

class GraphState(TypedDict):
    query: str
    entities: List[str]
    graph_data: Dict[str, Any]
    expanded_query: str
    results: Dict[str, Any]
    next_steps: List[str]

llm = LLMGateway(model_name='gpt-4o')

def entity_recognition_and_router(state: GraphState) -> Dict[str, Any]:
    # Implement entity recognition logic
    entities = ["entity1", "entity2"]  # Placeholder for entity recognition
    
    # Determine if we need to use graph DB
    if entities and need_graph_enrichment(entities):  # Implement need_graph_enrichment function
        return {
            "entities": entities,
            "next_steps": ["graph_rag_enrichment"]
        }
    else:
        return {
            "entities": entities,
            "next_steps": ["query_expansion"]
        }

def graph_rag_enrichment(state: GraphState) -> Dict[str, Any]:
    # Query graph database and extract relationships
    graph_data = {"related_entities": ["related1", "related2"]}  # Placeholder
    return {"graph_data": graph_data}

def query_expansion(state: GraphState) -> Dict[str, Any]:
    # Expand query based on entities and graph data
    expanded_terms = state['entities'] + state.get('graph_data', {}).get('related_entities', [])
    expanded_query = f"{state['query']} AND ({' OR '.join(expanded_terms)})"
    return {"expanded_query": expanded_query}

def query_optimization(state: GraphState) -> Dict[str, Any]:
    # Optimize the expanded query
    optimized_query = state["expanded_query"]  # Placeholder for optimization logic
    return {"optimized_query": optimized_query}

# ECT Chain (Earning Call Transcripts)
def create_ect_chain(llm, ect_schema_prompt: ChatPromptTemplate, ect_prompt: ChatPromptTemplate):
    # Implementation remains the same as before
    ...

ect_schema_prompt = ChatPromptTemplate.from_template("Your ECT_SCHEMA template here")
ect_prompt = ChatPromptTemplate.from_template("Your ECT_PROMPT template here")
ect_chain = create_ect_chain(llm, ect_schema_prompt, ect_prompt)

# Analyst Commentary Chain
def create_analyst_commentary_chain(llm, analyst_schema_prompt: ChatPromptTemplate, analyst_prompt: ChatPromptTemplate):
    # Similar structure to ect_chain, but for analyst commentary database
    ...

analyst_schema_prompt = ChatPromptTemplate.from_template("Your ANALYST_SCHEMA template here")
analyst_prompt = ChatPromptTemplate.from_template("Your ANALYST_PROMPT template here")
analyst_chain = create_analyst_commentary_chain(llm, analyst_schema_prompt, analyst_prompt)

def parallel_vector_search(state: GraphState) -> Dict[str, Any]:
    ect_results = ect_chain.invoke({"query": state["optimized_query"]})
    analyst_results = analyst_chain.invoke({"query": state["optimized_query"]})
    # Add more vector database searches as needed
    
    return {
        "results": {
            "ect": ect_results,
            "analyst": analyst_results
        }
    }

def result_aggregation(state: GraphState) -> Dict[str, Any]:
    # Aggregate results from different vector searches
    aggregated_results = {
        "ect": summarize_results(state["results"]["ect"]),
        "analyst": summarize_results(state["results"]["analyst"])
    }
    return {"aggregated_results": aggregated_results}

def context_integration(state: GraphState) -> Dict[str, Any]:
    # Integrate context from graph data and aggregated results
    context_integrated_results = {
        "results": state["aggregated_results"],
        "graph_context": state.get("graph_data", {})
    }
    return {"context_integrated_results": context_integrated_results}

def final_response_generation(state: GraphState) -> Dict[str, Any]:
    # Generate final response using LLM
    response_prompt = PromptTemplate.from_template(
        "Based on the following information, provide a comprehensive answer to the query: {query}\n\n"
        "Earning Call Transcript Summary: {ect_summary}\n"
        "Analyst Commentary Summary: {analyst_summary}\n"
        "Additional Context: {graph_context}\n\n"
        "Comprehensive Answer:"
    )
    
    final_response = llm(response_prompt.format(
        query=state["query"],
        ect_summary=state["context_integrated_results"]["results"]["ect"],
        analyst_summary=state["context_integrated_results"]["results"]["analyst"],
        graph_context=state["context_integrated_results"]["graph_context"]
    ))
    
    return {"final_response": final_response}

def feedback_collection(state: GraphState) -> Dict[str, Any]:
    # Collect feedback (this would typically be an external process)
    return {"feedback": "Placeholder feedback"}

# Build the graph
workflow = StateGraph(GraphState)

# Add nodes
workflow.add_node("initial_query", RunnableLambda(lambda x: x))
workflow.add_node("entity_recognition_and_router", entity_recognition_and_router)
workflow.add_node("graph_rag_enrichment", graph_rag_enrichment)
workflow.add_node("query_expansion", query_expansion)
workflow.add_node("query_optimization", query_optimization)
workflow.add_node("parallel_vector_search", parallel_vector_search)
workflow.add_node("result_aggregation", result_aggregation)
workflow.add_node("context_integration", context_integration)
workflow.add_node("final_response_generation", final_response_generation)
workflow.add_node("feedback_collection", feedback_collection)

# Add edges
workflow.add_edge("initial_query", "entity_recognition_and_router")
workflow.add_conditional_edges(
    "entity_recognition_and_router",
    lambda x: x["next_steps"],
    {
        "graph_rag_enrichment": "graph_rag_enrichment",
        "query_expansion": "query_expansion"
    }
)
workflow.add_edge("graph_rag_enrichment", "query_expansion")
workflow.add_edge("query_expansion", "query_optimization")
workflow.add_edge("query_optimization", "parallel_vector_search")
workflow.add_edge("parallel_vector_search", "result_aggregation")
workflow.add_edge("result_aggregation", "context_integration")
workflow.add_edge("context_integration", "final_response_generation")
workflow.add_edge("final_response_generation", "feedback_collection")
workflow.add_edge("feedback_collection", END)

# Set the entry point
workflow.set_entry_point("initial_query")

# Compile the graph
graph = workflow.compile()

# Run the graph
query = "What are the latest advancements in quantum computing?"
initial_state = {"query": query, "entities": [], "graph_data": {}, "expanded_query": "", "results": {}, "next_steps": []}
result = graph.invoke(initial_state)
print(f"Final result: {result['final_response']}")